# Fiber segmentation

In [1]:
import matplotlib.pylab as plt
%matplotlib inline
from itkwidgets import view, compare

from pathlib import Path
import numpy as np
import time

from skimage.morphology import skeletonize
from skimage.filters import threshold_otsu
import itk
import skan

import sys
import os
import warnings

import zarr
import xarray as xr

from dask.distributed import Client, LocalCluster
from fsspec.implementations.http import HTTPFileSystem
import dask_image.ndmeasure
import dask.array as da
import dask

The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.


In [2]:
# Use a Coiled.io Dask cloud cluster
# See https://github.com/dani-lbnl/SC20_pyHPC/blob/master/coiled/README.md
use_coiled = False
use_coiled = True

# If we are on the NERSC cluster and using the NERSC Scratch filesystem
def on_nersc():
    return 'SCRATCH' in os.environ
if on_nersc():
    os.chdir(os.environ['SCRATCH'])

In [3]:
if use_coiled:
    # Option 1: Coiled AWS cluster
    #
    # See https://github.com/dani-lbnl/SC20_pyHPC/blob/master/coiled/README.md
    #
    # You must first log into Coiled.
    import coiled
    # Set to re-use a running cluster when re-running the notebook. Listed at https://cloud.coiled.io/clusters.
    name = None
    cluster = coiled.Cluster(n_workers=100,
                             name=name,
                             configuration='thewtex/sc20-pyhpc',
                             software='thewtex/sc20-pyhpc',
                             worker_memory='6G')
    
    client = Client(cluster)
    
elif on_nersc():
    # Option 2: NERSC Cori Slurm dask-mpi cluster
    #
    # See SC20_pyHPC/nersc/README.md
    #
    # You must first start the dask cluster (Step 1).
    scheduler_file = os.path.join(os.environ["SCRATCH"], "scheduler.json")
    dask.config.config["distributed"]["dashboard"]["link"] = "{JUPYTERHUB_SERVICE_PREFIX}proxy/{host}:{port}/status"

    client = Client(scheduler_file=scheduler_file)
else:
    # Option 3: Local "cluster"
    #
    local_cluster = LocalCluster(n_workers=2, processes=False, memory_limit='6G')
    client = Client(local_cluster)
    client

Creating Cluster. This takes about a minute ...Checking environment images
Valid environment image found


In [4]:
if on_nersc():
    # Generated by fibers_to_xarray_zarr.ipynb
    store = './rec20160318_191511_232p3_2cm_cont__4097im_1500ms_ML17keV_6.zarr'
else:
    # We can access our API using fsspec's HTTPFileSystem
    fs = HTTPFileSystem()
    # The http mapper gives us a dict-like interface to the API
    store = fs.get_mapper("https://fiber-bed-zarr.netlify.com/rec20160318_191511_232p3_2cm_cont__4097im_1500ms_ML17keV_6.zarr")

In [5]:
ds = xr.open_zarr(store, consolidated=True)
ds

,Array,Chunk
Bytes,14.16 GB,262.14 kB
Shape,"(2160, 2560, 2560)","(64, 64, 64)"
Count,54401 Tasks,54400 Chunks
Type,uint8,numpy.ndarray


In [6]:
fibers = ds.rec20160318_191511_232p3_2cm_cont__4097im_1500ms_ML17keV_6
fibers

,Array,Chunk
Bytes,14.16 GB,262.14 kB
Shape,"(2160, 2560, 2560)","(64, 64, 64)"
Count,54401 Tasks,54400 Chunks
Type,uint8,numpy.ndarray


In [7]:
view(fibers[1000,:,:].values)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageUC2; pr…

In [8]:
view(fibers[950:1050,1000:1200,1000:1200].values)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageUC3; pr…

In [9]:
image = itk.image_view_from_array(fibers[950:1050,1000:1200,1000:1200].values.astype(np.float64))

In [10]:
sigma = 5.25
hessian = itk.hessian_recursive_gaussian_image_filter(image, sigma=sigma)
vesselness_filter = itk.Hessian3DToVesselnessMeasureImageFilter[itk.ctype('float')].New(hessian)
vesselness_filter.Update()
vesselness = vesselness_filter.GetOutput()

In [11]:
compare(image, vesselness, gradient_opacity=0.2, shadow=False, mode='z')

AppLayout(children=(HBox(children=(Label(value='Link:'), Checkbox(value=False, description='cmap'), Checkbox(v…

In [12]:
thresh = itk.otsu_threshold_image_filter(vesselness)
inverted = itk.invert_intensity_image_filter(thresh)
rescaled = itk.rescale_intensity_image_filter(inverted, ttype=(type(inverted), itk.Image[itk.UC, 3]))

In [13]:
view(image=rescaled)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageUC3; pr…

In [14]:
label, nlabels = dask_image.ndmeasure.label(np.asarray(rescaled)[50,:,:])
dask_image.ndmeasure.center_of_mass(label, label, da.unique(label).compute()).compute()

array([[         nan,          nan],
       [  3.42105263, 101.52631579],
       [  5.24731183, 167.94623656],
       [ 11.28282828, 189.62121212],
       [  6.5       ,  30.        ],
       [  8.        , 130.        ],
       [ 10.6       ,  14.2       ],
       [ 15.42168675,  64.27710843],
       [ 12.        ,  88.        ],
       [ 13.09090909, 151.        ],
       [ 15.82352941,  11.55882353],
       [ 16.25      ,  85.        ],
       [ 21.68333333,  38.6       ],
       [ 27.73333333,  52.06666667],
       [ 31.712     , 177.408     ],
       [ 29.9047619 ,   4.07142857],
       [ 28.17647059, 133.70588235],
       [ 30.94444444,  93.5       ],
       [ 33.48076923, 157.55769231],
       [ 36.37362637,  25.06593407],
       [ 41.35064935,  73.88961039],
       [ 37.        , 101.        ],
       [ 40.53846154, 150.92307692],
       [ 39.        , 199.        ],
       [ 42.76666667,  45.7       ],
       [ 46.26086957, 119.63768116],
       [ 46.        ,   9.5       ],
 

In [15]:
view(label_image=label.compute().astype(np.uint16))

Viewer(geometries=[], gradient_opacity=0.22, interpolation=False, point_sets=[], rendered_label_image=<itk.itk…

In [16]:
threshold_value = threshold_otsu(np.asarray(vesselness))
threshold_value

0.14946619

In [17]:
# To run on a smaller slab
# fiber_slab = fibers[950:1150,512:1024,512:1024]
fiber_slab = fibers
fiber_slab

,Array,Chunk
Bytes,14.16 GB,262.14 kB
Shape,"(2160, 2560, 2560)","(64, 64, 64)"
Count,54401 Tasks,54400 Chunks
Type,uint8,numpy.ndarray


In [18]:
fiber_slab = fiber_slab.chunk((100,128,128))
fiber_slab

,Array,Chunk
Bytes,14.16 GB,1.64 MB
Shape,"(2160, 2560, 2560)","(100, 128, 128)"
Count,127201 Tasks,8800 Chunks
Type,uint8,numpy.ndarray


In [19]:
def compute_vesselness(image_chunk, threshold=0.15):
    import itk
    
    image = itk.image_view_from_array(image_chunk.astype(np.float64))

    sigma = 5.25
    hessian = itk.hessian_recursive_gaussian_image_filter(image, sigma=sigma)
    vesselness_filter = itk.Hessian3DToVesselnessMeasureImageFilter[itk.ctype('float')].New(hessian)
    vesselness_filter.Update()
    vesselness = vesselness_filter.GetOutput()
    binary = itk.binary_threshold_image_filter(vesselness, upper_threshold=float(threshold))
    inverted = itk.invert_intensity_image_filter(binary)
    rescaled = itk.rescale_intensity_image_filter(inverted, ttype=(type(inverted), itk.Image[itk.UC, 3]))
    return np.asarray(rescaled)

In [20]:
seg = fiber_slab.data.map_overlap(compute_vesselness,
                                depth=32,
                                trim=True,
                                dtype=np.uint8,
                                threshold=threshold_value)

In [21]:
start = time.time()

seg = seg.compute()

elapsed = time.time() - start
print(elapsed, 'seconds')

distributed.comm.tcp - WARNING - Closing dangling stream in <TLS  local=tls://10.10.10.135:47814 remote=tls://ec2-3-129-244-192.us-east-2.compute.amazonaws.com:8786>
distributed.comm.tcp - WARNING - Closing dangling stream in <TLS  local=tls://10.10.10.135:47812 remote=tls://ec2-3-129-244-192.us-east-2.compute.amazonaws.com:8786>
distributed.comm.tcp - WARNING - Closing dangling stream in <TLS  local=tls://10.10.10.135:47824 remote=tls://ec2-3-129-244-192.us-east-2.compute.amazonaws.com:8786>
distributed.comm.tcp - WARNING - Closing dangling stream in <TLS  local=tls://10.10.10.135:47930 remote=tls://ec2-3-129-244-192.us-east-2.compute.amazonaws.com:8786>


AttributeError: 'FSMap' object has no attribute 'missing_exceptions'

In [ ]:
label, nlabels = dask_image.ndmeasure.label(seg)

In [ ]:
label, nlabels = dask.persist(label, nlabels)

In [ ]:
nlabels.compute()

In [ ]:
label_slice = label[50,:,:]

In [ ]:
label_slice_result = label_slice.compute()

In [ ]:
label_slice_result

In [ ]:
view(label_image=label_slice_result.astype(np.uint64))

In [ ]:
centers = dask_image.ndmeasure.center_of_mass(label_slice_result, label_slice_result, np.unique(l)).compute()

In [ ]:
centers.shape

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(centers[1:,:], columns=('X', 'Y'))

In [ ]:
df

In [ ]:
df.to_csv('./Results-slice1000-fiber_segmentation.csv')